In [16]:
import numpy as np
import pandas as pd
from scipy.optimize import fmin, minimize, fminbound

from sklearn.metrics import mean_squared_error, mean_absolute_error

import requests
import json

In [17]:
pd.set_option("display.max_colwidth", 0)

In [200]:
%connect_info

{
  "stdin_port": 53789, 
  "ip": "127.0.0.1", 
  "control_port": 37875, 
  "hb_port": 33716, 
  "signature_scheme": "hmac-sha256", 
  "key": "7bfdb66f-7ad6-405c-999b-89adf7252a4f", 
  "shell_port": 37704, 
  "transport": "tcp", 
  "iopub_port": 44846
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing /run/user/1014/jupyter/kernel-ec216574-84eb-4213-91db-0b3850df7f49.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [295]:
# Daily goal
# Protein-Fat-Carbs-Energy

y1 = 136; y2 = 95; y3 = 170
y0 = y1*4+y2*8+y3*4
y0 = 2000
y = np.array([y1, y2, y3, y0])

In [296]:
y

array([ 136,   95,  170, 2000])

In [297]:
# Relative MSE
def mse(A, B):
    return (((A - B)/A) ** 2).mean(axis=0)

In [298]:
apikey = '3jklPYmeuoykAvPVFSkTTXycJlxupddyBKtItMzg'

In [314]:
def search_by_name(ITEM, n=5):
    payload = {'api_key': apikey}
    r = requests.get('http://api.nal.usda.gov/ndb/search/?format=json&q={}&sort=r'.format(ITEM), params=payload)
    return pd.DataFrame(json.loads(r.text)['list']['item'][:n])

In [363]:
def food_by_id(id):
    payload = {'api_key': apikey,
               'nutrients': [208, 203, 205, 204]
              }
    r = requests.get('http://api.nal.usda.gov/ndb/nutrients/?ndbno={}'.format(id), params=payload)
    return json.loads(r.text)['report']['foods'][0]

In [348]:
def refresh(idx, replace_with=None):
    if replace_with is None:
        for i, k in enumerate(groups):
            v = foods[k]
            if i == idx:
                if k in ['Finfish and Shellfish Products', 'Beef Products', 'Lamb, Veal, and Game Products', 'Pork Products', 
                         'Poultry Products', 'Vegetables and Vegetable Products']:
                    v = [i for i in v if 'raw' in i['name']]
                p = np.random.choice(v, 1, False)

                print 'Replaced {} with \n{}'.format(picked[idx]['name'], p[0]['name'])
                picked[idx] = p[0]
    else:
        replacement = food_by_id(replace_with)
        print 'Replaced {} with {}'.format(picked[idx]['name'], replacement['name'])
        picked[idx] = replacement
    return [p['name'] for p in picked]

In [364]:
food_by_id(15152)

{u'measure': u'1.0 cup',
 u'name': u'Crustaceans, shrimp, mixed species, canned',
 u'ndbno': u'15152',
 u'nutrients': [{u'gm': 20.42,
   u'nutrient': u'Protein',
   u'nutrient_id': u'203',
   u'unit': u'g',
   u'value': u'26.14'},
  {u'gm': 1.36,
   u'nutrient': u'Total lipid (fat)',
   u'nutrient_id': u'204',
   u'unit': u'g',
   u'value': u'1.74'},
  {u'gm': 0.0,
   u'nutrient': u'Carbohydrate, by difference',
   u'nutrient_id': u'205',
   u'unit': u'g',
   u'value': u'0.00'},
  {u'gm': 100.0,
   u'nutrient': u'Energy',
   u'nutrient_id': u'208',
   u'unit': u'kcal',
   u'value': u'128'}],
 u'weight': 128.0}

In [307]:
payload = {'api_key': apikey,
           'lt': 'g'
           }
r = requests.get('http://api.nal.usda.gov/ndb/list?format=json', params=payload)

pd.DataFrame(json.loads(r.text)['list']['item']).sort_values('name')

,id,name,offset
0,3500,American Indian/Alaska Native Foods,0
1,0300,Baby Foods,1
2,1800,Baked Products,2
3,1300,Beef Products,3
4,1400,Beverages,4
5,0800,Breakfast Cereals,5
6,2000,Cereal Grains and Pasta,6
7,0100,Dairy and Egg Products,7
8,2100,Fast Foods,8
9,0400,Fats and Oils,9


In [378]:
search_by_name('pasta unenriched', 20)

,group,name,ndbno,offset
0,Cereal Grains and Pasta,"Pasta, dry, unenriched",20420,0
1,Cereal Grains and Pasta,"Pasta, cooked, unenriched, without added salt",20421,1
2,Cereal Grains and Pasta,"Pasta, cooked, unenriched, with added salt",20521,2
3,Cereal Grains and Pasta,"Pasta, whole grain, 51% whole wheat, remaining unenriched semolina, dry",20135,3
4,Cereal Grains and Pasta,"Pasta, whole grain, 51% whole wheat, remaining unenriched semolina, cooked",20136,4


In [320]:
foods = {}
for fg in ['0500', '0900', '1300', '1500', '1700', '1600', '1200', '1000', '1100', '2000']:
    print fg,
    payload = {'api_key': apikey,
               'fg': fg,
               'nutrients': [208, 203, 205, 204],
               'max': 1500,
               'subset': 1
              }
    r = requests.get('http://api.nal.usda.gov/ndb/nutrients/', params=payload)
    report = json.loads(r.text)['report']
    foods[str(report['groups'][0]['description'])] = report['foods']

0500 0900 1300 1500 1700 1600 1200 1000 1100 2000


In [321]:
foods.keys()

['Cereal Grains and Pasta',
 'Finfish and Shellfish Products',
 'Legumes and Legume Products',
 'Vegetables and Vegetable Products',
 'Beef Products',
 'Lamb, Veal, and Game Products',
 'Pork Products',
 'Nut and Seed Products',
 'Fruits and Fruit Juices',
 'Poultry Products']

In [338]:
picked = []
groups = []
for k, v in foods.items():
    if k in ['Vegetables and Vegetable Products']:
        size = 2
    elif k in ['Cereal Grains and Pasta']:
        size = 2
    else:
        size = 1
    if k in ['Finfish and Shellfish Products', 'Beef Products', 'Lamb, Veal, and Game Products', 'Pork Products', 
             'Poultry Products', 'Vegetables and Vegetable Products']:
        v = [i for i in v if 'raw' in i['name']]
    p = np.random.choice(v, size, False)
    picked.append(p)
    for i in range(size):
        groups.append(k)
picked = np.hstack(picked)
names = [p['name'] for p in picked]

In [396]:
names = refresh(2)

Replaced Fish, ocean perch, Atlantic, raw with 
Fish, tilapia, raw


In [422]:
for p in picked:
    for n in p['nutrients']:
        if n['gm'] == '--':
            n['gm'] = 0
            
x = np.array([[float(n[0]['gm']), float(n[1]['gm']), float(n[2]['gm']), float(n[3]['gm'])] 
              for n in [p['nutrients'] for p in picked]])
init = [2]*len(x)
bounds = tuple((0, None) for x in init)

#

# function to minimize
n_days = 5
def f(a):
    pred = np.dot(a, x)
    return mse(y*n_days, pred) #mean_absolute_error(y, pred)

result = minimize(f, init, method='SLSQP', bounds=bounds)
df = pd.DataFrame({'name': names, 'amount, g': np.round(result.x*100), 
                   'amount, oz': np.round(result.x*100*0.035274),
                   'amount, lbs': np.around(result.x*100*0.00220462, 2),
                   'group': groups,
                   'protein': np.round(result.x*x[:, 0]),
                   'fat': np.round(result.x*x[:, 1]),
                   'carbs': np.round(result.x*x[:, 2]),
                   'energy': np.round(result.x*x[:, 3]),
                  }
                  )
df_sum = pd.DataFrame(df.sum(axis=0)).T
df_sum[['name', 'group']] = [0, 0]
df = pd.concat([df, df_sum/5.0])

#

print y
print np.round(np.dot(result.x, x)/n_days)
df[['amount, g', 'amount, lbs', 'amount, oz' ,'group','name','protein','fat','carbs', 'energy']]
# print np.dot(np.where(result.x>0.1, result.x, 0), x)

[ 136   95  170 2000]
[  135.    94.   169.  2028.]


,"amount, g","amount, lbs","amount, oz",group,name,protein,fat,carbs,energy
0,611,1.35,22,Cereal Grains and Pasta,"Pasta, dry, unenriched",80,9,456,2267
1,331,0.73,12,Cereal Grains and Pasta,"Rice, white, long-grain, regular, cooked, unenriched, with salt",9,1,93,431
2,413,0.91,15,Finfish and Shellfish Products,"Fish, tilapia, raw",83,7,0,397
3,373,0.82,13,Legumes and Legume Products,"Beans, black turtle, mature seeds, cooked, boiled, without salt",31,1,91,485
4,252,0.56,9,Vegetables and Vegetable Products,"Cauliflower, green, raw",7,1,15,78
5,245,0.54,9,Vegetables and Vegetable Products,"Onions, raw",3,0,23,98
6,442,0.97,16,Beef Products,"Beef, ribeye cap steak, boneless, separable lean only, trimmed to 0"" fat, choice, raw",86,50,8,826
7,426,0.94,15,"Lamb, Veal, and Game Products","Lamb, Australian, imported, fresh, shoulder, blade, separable lean only, trimmed to 1/8"" fat, raw",81,39,0,699
8,419,0.92,15,Pork Products,"Pork, fresh, enhanced, loin, tenderloin, separable lean only, raw",85,9,0,444
9,612,1.35,22,Nut and Seed Products,"Seeds, sunflower seed kernels, toasted, without salt",105,348,126,3789
